In [5]:
import pandas as pd
import os
import datetime
import numpy as np
import gc

os.getcwd()

# To check about the 2019 weekly file path df week filter for the rolling later

'/home/jian/Projects/Big_Lots/Analysis/2019_Q2/ID_Migration'

In [6]:
prior_quarter_End=datetime.date(2019,5,4)
recent_complete_quarter_End=prior_quarter_End+datetime.timedelta(days=7*13)

print("prior_quarter_End",prior_quarter_End)
print("recent_complete_quarter_End",recent_complete_quarter_End)

lapsed_date_begin=prior_quarter_End-datetime.timedelta(days=(4*365+1+6))
print("lapsed_date_begin",lapsed_date_begin)


prior_quarter_End 2019-05-04
recent_complete_quarter_End 2019-08-03
lapsed_date_begin 2015-04-28


In [7]:
lapsed=pd.read_table("/home/jian/Projects/Big_Lots/Loyal_members/loyalty_sales_data/lapsed20140826_20170226/MediaStormLapsedCustDtl.txt",
                     sep=",",usecols=['customer_id_hashed','transaction_date'],dtype=str).drop_duplicates()
lapsed=lapsed[lapsed['transaction_date']>=str(lapsed_date_begin)]

print(lapsed['transaction_date'].min())
print(lapsed['transaction_date'].max())

2015-04-28
2017-02-26


In [8]:
chunksize_num = 10**7
filename='/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q1/crm_newscore_0922/combinedtransactions_0922.csv'
dftrans_before_20180922=pd.DataFrame()
count_i=0

for chunk in pd.read_csv(filename, chunksize=chunksize_num,dtype=str,usecols=['customer_id_hashed','transaction_date']):
    chunk = chunk.drop_duplicates()
    chunk=chunk[chunk['transaction_date']>=str(lapsed_date_begin)]
    dftrans_before_20180922=dftrans_before_20180922.append(chunk)
    count_i+=1
    print(count_i,datetime.datetime.now())
dftrans_before_20180922=dftrans_before_20180922.drop_duplicates()    

1 2019-08-07 21:31:26.616446
2 2019-08-07 21:31:53.074990
3 2019-08-07 21:32:17.830312
4 2019-08-07 21:32:45.387263
5 2019-08-07 21:33:14.569063
6 2019-08-07 21:33:43.638857
7 2019-08-07 21:34:18.338592
8 2019-08-07 21:34:54.313731
9 2019-08-07 21:35:29.273823
10 2019-08-07 21:36:08.936892
11 2019-08-07 21:36:44.150872
12 2019-08-07 21:37:21.970902
13 2019-08-07 21:38:00.639112
14 2019-08-07 21:38:38.509468
15 2019-08-07 21:39:15.204128


In [9]:
all_rewards_most_recent=dftrans_before_20180922.append(lapsed)
all_rewards_most_recent=all_rewards_most_recent.sort_values(["customer_id_hashed","transaction_date"],ascending=[True,False])
all_rewards_most_recent=all_rewards_most_recent.drop_duplicates(['customer_id_hashed'])
print(all_rewards_most_recent.shape)

del chunk
del dftrans_before_20180922
gc.collect()

(22943190, 2)


84

In [10]:
def recursive_file_gen(root_path):
    for root, dirs, files in os.walk(root_path):
        for file in files:
            yield os.path.join(root,file)

In [11]:
# Before the end of 2019Q1           
historical_daily_data_folder="/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/"
historical_daily_data_list=list(recursive_file_gen(historical_daily_data_folder))
historical_daily_data_list=[x for x in historical_daily_data_list if (".txt" in x) & ("DailySales" in x)]
historical_daily_df=pd.DataFrame({"file_path":historical_daily_data_list})
historical_daily_df['week_end_dt']=historical_daily_df['file_path'].apply(lambda x: x.split(".")[0].split("MediaStormDailySalesHistory")[1])
historical_daily_df['week_end_dt']=historical_daily_df['week_end_dt'].apply(lambda x: datetime.datetime.strptime(x,"%Y%m%d").date())
historical_daily_df=historical_daily_df[historical_daily_df['week_end_dt']<=datetime.date(2019,5,4)] # hard-code due to the 1st week of daily in item available
historical_daily_df=historical_daily_df[historical_daily_df['week_end_dt']>datetime.date(2018,9,22)] # hard-code
historical_daily_df=historical_daily_df.sort_values("week_end_dt")
print(historical_daily_df.shape)
historical_daily_df.tail(3)

(20, 2)


,file_path,week_end_dt
24,/home/jian/BigLots/hist_daily_data_itemlevel_d...,2019-01-26
25,/home/jian/BigLots/hist_daily_data_itemlevel_d...,2019-02-02
26,/home/jian/BigLots/hist_daily_data_itemlevel_d...,2019-02-09


In [12]:
prior_quarter_End

datetime.date(2019, 5, 4)

In [13]:
recent_complete_quarter_End

datetime.date(2019, 8, 3)

In [14]:
df_historical_daily_20180929_to_before_Q=historical_daily_df[historical_daily_df['week_end_dt']<=prior_quarter_End]
df_historical_daily_in_recent_Q=historical_daily_df[(historical_daily_df['week_end_dt']>prior_quarter_End) & (historical_daily_df['week_end_dt']<=recent_complete_quarter_End)]
print(df_historical_daily_20180929_to_before_Q.shape)
print(df_historical_daily_in_recent_Q.shape)


(20, 2)
(0, 2)


In [15]:
weekly_2019_daily_df=list(recursive_file_gen("/home/jian/BigLots/"))

weekly_2019_daily_df=[x for x in weekly_2019_daily_df if "2018" not in x]
weekly_2019_daily_df=[x for x in weekly_2019_daily_df if "2017" not in x]
weekly_2019_daily_df=[x for x in weekly_2019_daily_df if "2016" not in x]
weekly_2019_daily_df=[x for x in weekly_2019_daily_df if "2016" not in x]
weekly_2019_daily_df=[x for x in weekly_2019_daily_df if "Historical" not in x]
weekly_2019_daily_df=[x for x in weekly_2019_daily_df if "hist_daily_data_itemlevel_decompressed" not in x]
weekly_2019_daily_df=[x for x in weekly_2019_daily_df if ".txt" in x]
weekly_2019_daily_df=[x for x in weekly_2019_daily_df if "aily" in x]
weekly_2019_daily_df=pd.DataFrame({"file_path":weekly_2019_daily_df})
weekly_2019_daily_df['week_end_dt']=weekly_2019_daily_df['file_path'].apply(lambda x: x.split("/MediaStorm_")[1][:10])

weekly_2019_daily_df['week_end_dt']=weekly_2019_daily_df['week_end_dt'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
print(weekly_2019_daily_df.shape)
weekly_2019_daily_df.head(2)

weekly_2019_daily_df_in_recent_Q=weekly_2019_daily_df[(weekly_2019_daily_df['week_end_dt']>prior_quarter_End) & (weekly_2019_daily_df['week_end_dt']<=recent_complete_quarter_End)]
# Change to > prior_quarter_End for the next quarter
print(weekly_2019_daily_df_in_recent_Q.shape)


weekly_2019_daily_df_before_Q=weekly_2019_daily_df[weekly_2019_daily_df['week_end_dt']<=prior_quarter_End]
weekly_2019_daily_df_before_Q=weekly_2019_daily_df_before_Q[weekly_2019_daily_df_before_Q['week_end_dt']>df_historical_daily_20180929_to_before_Q['week_end_dt'].max()]
print(weekly_2019_daily_df_before_Q.shape) # Q1 should be 0, and Q2&after not


(31, 2)
(13, 2)
(12, 2)


In [16]:
weekly_2019_daily_df_in_recent_Q.shape

(13, 2)

In [17]:
weekly_2019_daily_df_before_Q.shape

(12, 2)

In [18]:
df_all_daily_files_20190922_before_quarter=df_historical_daily_20180929_to_before_Q.append(weekly_2019_daily_df_before_Q)

df_all_daily_files_in_recent_quarter=df_historical_daily_in_recent_Q.append(weekly_2019_daily_df_in_recent_Q)


In [19]:
print(df_all_daily_files_20190922_before_quarter.shape)
df_all_daily_files_20190922_before_quarter.sort_values("week_end_dt")

(32, 2)


,file_path,week_end_dt
7,/home/jian/BigLots/hist_daily_data_itemlevel_d...,2018-09-29
8,/home/jian/BigLots/hist_daily_data_itemlevel_d...,2018-10-06
9,/home/jian/BigLots/hist_daily_data_itemlevel_d...,2018-10-13
10,/home/jian/BigLots/hist_daily_data_itemlevel_d...,2018-10-20
11,/home/jian/BigLots/hist_daily_data_itemlevel_d...,2018-10-27
12,/home/jian/BigLots/hist_daily_data_itemlevel_d...,2018-11-03
13,/home/jian/BigLots/hist_daily_data_itemlevel_d...,2018-11-10
14,/home/jian/BigLots/hist_daily_data_itemlevel_d...,2018-11-17
15,/home/jian/BigLots/hist_daily_data_itemlevel_d...,2018-11-24
16,/home/jian/BigLots/hist_daily_data_itemlevel_d...,2018-12-01


In [20]:
print(df_all_daily_files_in_recent_quarter.shape)
df_all_daily_files_in_recent_quarter.sort_values("week_end_dt")

(13, 2)


,file_path,week_end_dt
18,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-05-11
19,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-05-18
20,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-05-25
21,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-06-01
22,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-06-08
23,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-06-15
24,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-06-22
25,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-06-29
26,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-07-06
27,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-07-13


In [21]:
rew_df_after_20180929_before_quarter=pd.DataFrame()
rew_df_after_recent_quarter_only=pd.DataFrame()


for file in df_all_daily_files_20190922_before_quarter['file_path'].tolist():
    df=pd.read_table(file,sep="|",dtype=str,usecols=["customer_id_hashed","transaction_dt"]).drop_duplicates().rename(columns={"transaction_dt":"transaction_date"})
    df=df[~pd.isnull(df['customer_id_hashed'])]
    rew_df_after_20180929_before_quarter=rew_df_after_20180929_before_quarter.append(df)
    print(file,datetime.datetime.now())

for file in df_all_daily_files_in_recent_quarter['file_path'].tolist():
    df=pd.read_table(file,sep="|",dtype=str,usecols=["customer_id_hashed","transaction_dt"]).drop_duplicates().rename(columns={"transaction_dt":"transaction_date"})
    df=df[~pd.isnull(df['customer_id_hashed'])]
    rew_df_after_recent_quarter_only=rew_df_after_recent_quarter_only.append(df)
    print(file,datetime.datetime.now())


/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20180929.txt 2019-08-07 21:58:49.860162
/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181006.txt 2019-08-07 21:59:18.376651
/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181013.txt 2019-08-07 21:59:39.915444
/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181020.txt 2019-08-07 22:00:01.396450
/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181027.txt 2019-08-07 22:00:25.081618
/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181103.txt 2019-08-07 22:00:48.920847
/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181110.txt 2019-08-07 22:01:14.973013
/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181117.txt 2019-08-07 22:01:44.741439


In [22]:
all_rewards_most_recent=all_rewards_most_recent.append(rew_df_after_20180929_before_quarter)
all_rewards_most_recent=all_rewards_most_recent.sort_values(["customer_id_hashed","transaction_date"],ascending=[True,False])
all_rewards_most_recent=all_rewards_most_recent.drop_duplicates(['customer_id_hashed'])
all_rewards_most_recent['transaction_date_before_the_Quarter']=all_rewards_most_recent['transaction_date']

all_rewards_most_recent_After=all_rewards_most_recent[['customer_id_hashed','transaction_date']].append(rew_df_after_recent_quarter_only)
all_rewards_most_recent_After=all_rewards_most_recent_After.sort_values(["customer_id_hashed","transaction_date"],ascending=[True,False])
all_rewards_most_recent_After=all_rewards_most_recent_After.drop_duplicates(['customer_id_hashed']).rename(columns={"transaction_date":"transaction_date_after_the_Quarter"})

all_rewards_most_recent=pd.merge(all_rewards_most_recent,all_rewards_most_recent_After,on="customer_id_hashed",how="outer")
del all_rewards_most_recent_After

gc.collect()

79

In [23]:
rew_df_after_20180929_before_quarter.tail(2)

,transaction_date,customer_id_hashed
12959879,2019-05-03,da09831b5743ecd5b42ffc59c4155df6ff2ea8a320be94...
12960022,2019-05-04,fd437b6358f7476efbd4af9c423a7ca7cff605f1a4e054...


In [24]:
rew_df_after_recent_quarter_only.tail(2)

,transaction_date,customer_id_hashed
12889781,2019-07-31,44bc77f7dde6f9207299a4c167da50d358571c2c9251eb...
12889786,2019-08-02,c7e952dcfccd75a749f43482de2ae83aee7291047ca21e...


In [25]:
all_rewards_most_recent['transaction_date_before_the_Quarter']=all_rewards_most_recent['transaction_date_before_the_Quarter'].fillna("2099-12-31")
all_rewards_most_recent['transaction_date_before_the_Quarter']=pd.to_datetime(all_rewards_most_recent['transaction_date_before_the_Quarter'],format='%Y-%m-%d')
all_rewards_most_recent['Days_to_pre_Quarter_End']=prior_quarter_End-all_rewards_most_recent['transaction_date_before_the_Quarter']
# Filled the na before with a later adte 2019-12-31


all_rewards_most_recent['transaction_date_after_the_Quarter']=pd.to_datetime(all_rewards_most_recent['transaction_date_after_the_Quarter'],format='%Y-%m-%d')
all_rewards_most_recent['Days_to_recentt_Quarter_End']=recent_complete_quarter_End-all_rewards_most_recent['transaction_date_after_the_Quarter']
print(datetime.datetime.now())

2019-08-07 22:43:03.838910


In [26]:
all_rewards_most_recent['Month_to_pre_Quarter_End']=all_rewards_most_recent['Days_to_pre_Quarter_End'].apply(lambda x: x.days/(365.25/12))
all_rewards_most_recent['Month_to_recent_Quarter_End']=all_rewards_most_recent['Days_to_recentt_Quarter_End'].apply(lambda x: x.days/(365.25/12))
print(datetime.datetime.now())

2019-08-07 22:53:15.883422


In [27]:
all_rewards_most_recent['Group_before_the_Quarter']=np.where((all_rewards_most_recent['Month_to_pre_Quarter_End']>12) & (all_rewards_most_recent['Month_to_pre_Quarter_End']<=48),"Lapsed_13_48",
                                                   np.where(all_rewards_most_recent['Month_to_pre_Quarter_End']>48,"WD_48+",
                                                            np.where((all_rewards_most_recent['Month_to_pre_Quarter_End']>=0) & (all_rewards_most_recent['Month_to_pre_Quarter_End']<=12),"Active",
                                                                    np.where(all_rewards_most_recent['Month_to_pre_Quarter_End']<0,"NotAvailable_Before_Quarter","NaN")
                                                                    )
                                                           )
                                                   )
all_rewards_most_recent['Group_after_the_Quarter']=np.where((all_rewards_most_recent['Month_to_recent_Quarter_End']>12) & (all_rewards_most_recent['Month_to_recent_Quarter_End']<=48),"Lapsed_13_48",
                                                   np.where(all_rewards_most_recent['Month_to_recent_Quarter_End']>48,"WD_48+",
                                                            np.where((all_rewards_most_recent['Month_to_recent_Quarter_End']>=0) & (all_rewards_most_recent['Month_to_recent_Quarter_End']<=12),"Active",
                                                                    np.where(all_rewards_most_recent['Month_to_recent_Quarter_End']<0,"NotAvailable_Before_Q4","NaN")
                                                                    )
                                                           )
                                                   )

In [28]:
all_rewards_most_recent['Group_before_the_Quarter'].unique().tolist()

['Active', 'Lapsed_13_48', 'WD_48+', 'NotAvailable_Before_Quarter']

In [29]:
all_rewards_most_recent['Group_after_the_Quarter'].unique()

array(['Active', 'Lapsed_13_48', 'WD_48+'], dtype=object)

In [30]:
RecentQuarter_sales_by_id=pd.DataFrame()

for file in df_all_daily_files_in_recent_quarter['file_path'].tolist():
    df=pd.read_table(file,sep="|",dtype=str,
                     usecols=['location_id','transaction_dt','transaction_id','customer_id_hashed','item_transaction_amt'])
    df=df[~pd.isnull(df['customer_id_hashed'])]
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    df_sales=df.groupby(['customer_id_hashed'])['item_transaction_amt'].sum().to_frame().reset_index().rename(columns={"item_transaction_amt":"sales_recent_Quarter"})
    
    df_trans=df[['location_id','transaction_dt','transaction_id','customer_id_hashed']].drop_duplicates()
    df_trans=df_trans.groupby(["customer_id_hashed"])['transaction_id'].count().to_frame().reset_index().rename(columns={"transaction_id":"trans_recent_Quarter"})
    
    df=pd.merge(df_sales,df_trans,on="customer_id_hashed",how="outer")
    
    RecentQuarter_sales_by_id=RecentQuarter_sales_by_id.append(df)
    print(file,datetime.datetime.now())
RecentQuarter_sales_by_id=RecentQuarter_sales_by_id.groupby("customer_id_hashed")['sales_recent_Quarter','trans_recent_Quarter'].sum().reset_index()
RecentQuarter_sales_by_id['Recent_Quarter_Shopping_Group']="Recent_Quarter_Shopped"

/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-05-11/MediaStormDailySales20190514-121223-405.txt 2019-08-07 22:55:05.818165
/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-05-18/MediaStormDailySales20190521-111232-391.txt 2019-08-07 22:55:55.616834
/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-05-25/MediaStormDailySales20190528-111211-863.txt 2019-08-07 22:56:52.154281
/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-06-01/MediaStormDailySales20190604-111936-950.txt 2019-08-07 22:57:44.010926
/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-06-08/MediaStormDailySales20190611-122013-442.txt 2019-08-07 22:58:39.443402
/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-06-15/MediaStormDailySales20190618-112500-846.txt 2019-08-07 22:59:33.133105
/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-06-22/MediaStormDailySales20190625-111927-257.txt 2019-08-07 23:00:26.820045
/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-06-29/MediaStormDailySales20190702-113523-928.txt 2019-08-07 23:01:23

In [31]:
all_rewards_most_recent=pd.merge(all_rewards_most_recent,RecentQuarter_sales_by_id,on="customer_id_hashed",how="outer")
all_rewards_most_recent.shape

(28427573, 13)

In [ ]:
# Stop here

In [32]:
Recent_Quarter_new_sign_ups_files=list(recursive_file_gen("/home/jian/BigLots/"))

Recent_Quarter_new_sign_ups_files=[x for x in Recent_Quarter_new_sign_ups_files if "Master" in x]
Recent_Quarter_new_sign_ups_files=[x for x in Recent_Quarter_new_sign_ups_files if "2018" not in x]
Recent_Quarter_new_sign_ups_files=[x for x in Recent_Quarter_new_sign_ups_files if "2017" not in x]
Recent_Quarter_new_sign_ups_files=[x for x in Recent_Quarter_new_sign_ups_files if "2016" not in x]
Recent_Quarter_new_sign_ups_files=[x for x in Recent_Quarter_new_sign_ups_files if ".txt" in x]

Recent_Quarter_new_sign_ups_files_df=pd.DataFrame({"file_path":Recent_Quarter_new_sign_ups_files})
Recent_Quarter_new_sign_ups_files_df['week_end_dt']=Recent_Quarter_new_sign_ups_files_df['file_path'].apply(lambda x: x.split("/MediaStorm_")[1][:10])
Recent_Quarter_new_sign_ups_files_df=Recent_Quarter_new_sign_ups_files_df[Recent_Quarter_new_sign_ups_files_df['week_end_dt']<=str(recent_complete_quarter_End)]
Recent_Quarter_new_sign_ups_files_df=Recent_Quarter_new_sign_ups_files_df[Recent_Quarter_new_sign_ups_files_df['week_end_dt']>str(prior_quarter_End)]
print(Recent_Quarter_new_sign_ups_files_df.shape)
Recent_Quarter_new_sign_ups_files_df.head(2)

(13, 2)


,file_path,week_end_dt
16,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-05-11
17,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-05-18


In [33]:
df_new_sign_the_gap="/home/jian/BigLots/New_Sing_Ups_2018_Fiscal_Year/MediaStorm Rewards Master P4 2019 - no transaction info.zip"
df_new_sign_the_gap=pd.read_table(df_new_sign_the_gap,sep="|",usecols=['customer_id_hashed','sign_up_date'])
df_new_sign_the_gap.head(2)

,customer_id_hashed,sign_up_date
0,a30cc90d9c489ad7f0ab223ba7ea0917dcc74e0bc7ff51...,2019-05-13
1,ba076e9261ef546379354f54e5a9f37a1fd570342a5635...,2019-05-16


In [35]:
df_new_sign_the_gap['sign_up_date'].min()

'2019-05-05'

In [36]:
all_new_sign_ups=pd.DataFrame()
for file in Recent_Quarter_new_sign_ups_files_df['file_path'].unique().tolist():
    df=pd.read_table(file,sep="|",usecols=['customer_id_hashed','sign_up_date'])
    all_new_sign_ups=all_new_sign_ups.append(df)
all_new_sign_ups=all_new_sign_ups.append(df_new_sign_the_gap)
all_new_sign_ups=all_new_sign_ups.sort_values(['customer_id_hashed','sign_up_date'])
all_new_sign_ups=all_new_sign_ups.drop_duplicates("customer_id_hashed")
print(all_new_sign_ups.shape)

all_new_sign_ups['NewRewards']="Recent_Quarter_New_Sign_Ups"
all_new_sign_ups=all_new_sign_ups.drop_duplicates("customer_id_hashed")

(1614908, 2)


In [37]:
all_rewards_most_recent=pd.merge(all_rewards_most_recent,all_new_sign_ups,on="customer_id_hashed",how="outer")
all_rewards_most_recent['NewRewards']=all_rewards_most_recent['NewRewards'].fillna("Old_Rewards_Members")

all_rewards_most_recent['sales_recent_Quarter']=all_rewards_most_recent['sales_recent_Quarter'].fillna(0)
all_rewards_most_recent['trans_recent_Quarter']=all_rewards_most_recent['trans_recent_Quarter'].fillna(0)

all_rewards_most_recent['Recent_Quarter_Shopping_Group']=all_rewards_most_recent['Recent_Quarter_Shopping_Group'].fillna("Recent_Quarter_No_Shop")

all_rewards_most_recent.shape


(28897174, 15)

In [38]:
output_id_count=all_rewards_most_recent.groupby(['Group_before_the_Quarter','Group_after_the_Quarter','NewRewards','Recent_Quarter_Shopping_Group'])['customer_id_hashed'].count().to_frame().reset_index().rename(columns={"customer_id_hashed":"id_count"})
output_id_Q4_sales=all_rewards_most_recent.groupby(['Group_before_the_Quarter','Group_after_the_Quarter','NewRewards','Recent_Quarter_Shopping_Group'])['sales_recent_Quarter','trans_recent_Quarter'].sum().reset_index()

output=pd.merge(output_id_count,output_id_Q4_sales,on=['Group_before_the_Quarter','Group_after_the_Quarter','NewRewards','Recent_Quarter_Shopping_Group'],how="outer")
output


,Group_before_the_Quarter,Group_after_the_Quarter,NewRewards,Recent_Quarter_Shopping_Group,id_count,sales_recent_Quarter,trans_recent_Quarter
0,Active,Active,Old_Rewards_Members,Recent_Quarter_No_Shop,9546619,0.000000e+00,0.0
1,Active,Active,Old_Rewards_Members,Recent_Quarter_Shopped,6576945,5.933058e+08,16528695.0
2,Active,Active,Recent_Quarter_New_Sign_Ups,Recent_Quarter_No_Shop,19,0.000000e+00,0.0
3,Active,Active,Recent_Quarter_New_Sign_Ups,Recent_Quarter_Shopped,41,8.970590e+03,152.0
4,Active,Lapsed_13_48,Old_Rewards_Members,Recent_Quarter_No_Shop,1596764,0.000000e+00,0.0
5,Active,Lapsed_13_48,Recent_Quarter_New_Sign_Ups,Recent_Quarter_No_Shop,1,0.000000e+00,0.0
6,Lapsed_13_48,Active,Old_Rewards_Members,Recent_Quarter_Shopped,666542,5.207722e+07,1012331.0
7,Lapsed_13_48,Lapsed_13_48,Old_Rewards_Members,Recent_Quarter_No_Shop,8419112,0.000000e+00,0.0
8,Lapsed_13_48,WD_48+,Old_Rewards_Members,Recent_Quarter_No_Shop,110952,0.000000e+00,0.0
9,NotAvailable_Before_Quarter,Active,Old_Rewards_Members,Recent_Quarter_Shopped,358838,3.386871e+07,554246.0


In [39]:
os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q2/ID_Migration'

In [40]:
output['Recent_Quarteer_Sale_per_ID']=output['sales_recent_Quarter']/output['id_count']
output['Recent_Quarteer_Sale_per_ID']=output['Recent_Quarteer_Sale_per_ID'].apply(lambda x: np.round(x,2))

output['inclusion/exclusion']=np.where((output['Group_before_the_Quarter']!="NotAvailable_Before_Quarter") & (output['NewRewards']=="Recent_Quarter_New_Sign_Ups"),"Exclusion","Inclusion")


In [ ]:
output.to_csv("/home/jian/Projects/Big_Lots/Analysis/2019_Q2/ID_Migration/BL_2019Q2_rewards_migration_output_summary_JL_"+str(datetime.datetime.now().date())+".csv",index=False)



In [ ]:
all_rewards_most_recent.to_csv("/home/jian/Projects/Big_Lots/Analysis/2019_Q2/ID_Migration/BL_2019Q2_rewards_migration_rewards_data_JL_"+str(datetime.datetime.now().date())+".csv",index=False)


In [ ]:
all_rewards_most_recent.shape

In [ ]:
all_rewards_most_recent['customer_id_hashed'].nunique()